<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/4_Aprendizaje_no_supervisado/1_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🔵🟢🟠 **Análisis de Clusters**

**Objetivos de Aprendizaje**

Al finalizar este notebook, serás capaz de:

1.  **Entender la intuición económica** detrás del análisis de clustering y su aplicación en la segmentación de mercados.
2. **Implementar** dos algoritmos de agrupamiento fundamentales: K-Means y K-prototypes.
3. **Determinar el número óptimo de clusters** utilizando técnicas como el Método del Codo y el Coeficiente de Silueta.
4. **Interpretar los resultados** de los algoritmos de clustering para extraer conclusiones de negocio y económicas valiosas.

**Introducción**

Imagina que eres el gerente de un centro comercial. Tienes datos sobre tus clientes: cuánto ganan y cuánto gastan. Quieres lanzar campañas de marketing, pero enviar el mismo mensaje a todos es ineficiente. A un cliente que gana mucho y gasta poco (un ahorrador) no le interesará la misma promoción que a un cliente que gana poco pero gasta mucho (un entusiasta de las ofertas).

El análisis de clustering es una técnica de machine learning no supervisado que nos permite hacer exactamente esto: descubrir grupos (o clusters) naturales en nuestros datos sin que nos digan previamente cuáles son esos grupos. El algoritmo "mira" los datos y agrupa a los clientes que se parecen entre sí.

## Importar librerías

In [ ]:
# Importación de librerías estándar
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importación de modelos y métricas de Scikit-Learn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## Mejorar visualización de dataframes y gráficos

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## Cargar el dataset

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/2025_ii_curso_ia_economia/datasets'

In [ ]:
# Para establecer el directorio de los archivos
os.chdir(path)

In [ ]:
df = pd.read_csv('Mall_Customers.csv')
df

In [ ]:
# Renombrar columnas para facilidad de uso
df.rename(columns={
    'Annual Income (k$)': 'Annual_Income',
    'Spending Score (1-100)': 'Spending_Score'
}, inplace=True)

In [ ]:
df.info()

In [ ]:
print("\nEstadísticas Descriptivas:")
print(df.describe())

Los datos están limpios (no hay nulos) y listos para el análisis.

## Visualización: ¿Podemos "Ver" los Clusters?

La mejor forma de empezar es visualizando la relación entre el ingreso anual y la puntuación de gasto.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Annual_Income', y='Spending_Score', data=df)
plt.title('Ingreso Anual vs. Puntuación de Gasto')
plt.xlabel('Ingreso Anual (en miles de $)')
plt.ylabel('Puntuación de Gasto (1-100)')
plt.show()

A simple vista, podemos identificar 5 grupos de clientes. Esto es una gran ventaja, ya que nos da una hipótesis clara para probar con nuestros algoritmos.

## Preprocesamiento: preparando los datos para el Clustering

Para este análisis, nos enfocaremos en las dos variables que visualizamos.

Debido a que los algoritmos de clustering se basan en distancias, se deben estandarizar las variables para que ninguna domine a la otra solo por su escala.

In [ ]:
# Seleccionamos las características para el clustering
X = df[['Annual_Income', 'Spending_Score']]

# Estandarizamos las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertimos a DataFrame para facilidad de uso posterior
X_scaled_df = pd.DataFrame(X_scaled, columns=['Annual_Income', 'Spending_Score'])
X_scaled_df


In [ ]:
sns.pairplot(X)
plt.suptitle('Pairplot de Datos Originales', y=1.02)
plt.show()

In [ ]:
# Generar un pairplot de los datos escalados
sns.pairplot(X_scaled_df)
plt.suptitle('Pairplot de Datos Estandarizados', y=1.02) # Añadir un título general
plt.show()

## K-Means

- K-Means es el algoritmo de clustering más popular por su simplicidad y eficiencia.

- Divide una muestra n en K clases.

- Solo se aplica a variables numéricas.

Pasos que sigue el algoritmo:

1. El algoritmo asigna aleatoriamente 'k' puntos como centros de cada cluster (centroides).

2. Asigna cada observación al centroide más cercano.

3. Recalcula la posición de cada centroide como el promedio de todos los puntos asignados a él.

4. Repite los pasos 2 y 3 hasta que los centroides ya no se mueven o hasta que se alcance el número de iteraciones que le indiquemos.



In [ ]:
from IPython.display import YouTubeVideo

# ID del video: https://www.youtube.com/watch?v=mICySHB0fh4
YouTubeVideo('mICySHB0fh4', width=800, height=450)

**K-Means utiliza la Distancia Euclidiana**

Es la que todos conocemos de la geometría. La distancia más corta entre dos puntos: la hipotenusa de un triángulo.

Fórmula:

$$d(p, q) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2}$$

p y q: Son nuestros dos puntos (Cliente A y Cliente B).

i: Es un índice que representa cada una de las dimensiones que estamos considerando (dimensión 1: Ingreso, dimensión 2: Gasto, dimensión 3: Edad, etc.).

(pi−qi): Es la diferencia entre los dos puntos a lo largo de una sola dimensión i.

Ventajas:

- Muy intuitiva y fácil de entender.
- Funciona excelentemente cuando los clusters son compactos y esféricos.

Desventajas:

- La Maldición de la Dimensionalidad: A medida que añadimos más variables (dimensiones), la distancia euclidiana pierde significado. En un espacio de muchas dimensiones, la distancia entre cualquier par de puntos tiende a ser muy similar, dificultando la formación de clusters densos.
- Sensible a la escala de las variables (por eso siempre estandarizamos).

Casos de Uso:
- Segmentación de clientes.
- Agrupación de acciones por retorno y volatilidad.
- Agrupación de establecimientos según ventas y número de clientes.

### Aplicar K-Means y visualizar los resultados

Ahora, aplicamos K-Means con k=5, según lo que vimos en la gráfica.

In [ ]:
# Aplicar K-Means con k=5
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=42)
clusters_kmeans = kmeans.fit_predict(X_scaled_df)
clusters_kmeans


**Nota:**

- init='k-means++': Es una forma 'inteligente' de elegir los centroides iniciales. En lugar de ponerlos al azar, este método trata de elegirlos lo más separados posible entre sí. Esto hace que el algoritmo converja mucho más rápido y a un mejor resultado. Es la opción por defecto y casi siempre la mejor.

- n_init=10: K-Means puede a veces quedarse 'atrapado' en una solución subóptima dependiendo de dónde empezaron los centroides. Este parámetro le dice a Scikit-Learn: 'Corre todo el algoritmo 10 veces desde el principio con diferentes centroides iniciales, y al final quédate con el mejor resultado de los 10'. Esto aumenta la probabilidad de encontrar la mejor solución posible.

- max_iter: Es el número máximo de iteraciones (ciclos de "asignar puntos -> actualizar centroides") que el algoritmo realizará en una sola ejecución. Es una salvaguarda, ya que K-Means suele converger muy rápido (en menos de 100 iteraciones). Este límite evita que el algoritmo se ejecute indefinidamente si por alguna razón no converge. El valor por defecto es de 300 que es más que suficiente para la gran mayoría de los problemas. Rara vez necesitarás cambiarlo.

In [ ]:
# Añadir los clusters al DataFrame original
df['Cluster_KMeans'] = clusters_kmeans
df

In [ ]:
# Ver las coordenadas de los centroides
kmeans.cluster_centers_

In [ ]:
# Visualizar los clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Annual_Income', y='Spending_Score', hue='Cluster_KMeans', data=df, palette='viridis', s=100)

# Visualizar los centroides
centroids = scaler.inverse_transform(kmeans.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='Centroides')

plt.title('Segmentación de Clientes con K-Means')
plt.xlabel('Ingreso Anual (en miles de $)')
plt.ylabel('Puntuación de Gasto (1-100)')
plt.legend()
plt.show()

### Determinar el Número Óptimo de Clusters (k)

#### Inercia

Cuando entrenamos un modelo K-Means, el resultado tiene un atributo llamado inertia_. Este número no es solo un resultado técnico; **es el objetivo central que el algoritmo K-Means intenta minimizar**.

En términos sencillos, **la inercia mide qué tan internamente coherentes y compactos son los clusters**. Un valor de inercia más bajo significa que los clusters son más densos y están mejor definidos, ya que los puntos de datos están, en promedio, más cerca de sus respectivos centroides.

Se calcula de la siguiente manera:

1. Para cada punto de dato, medimos la distancia al centroide.
2. Elevamos esa distancia al cuadrado. (Elevar al cuadrado penaliza más fuertemente a los puntos que están muy lejos).
3. Sumamos los resultados de todos los puntos.

El objetivo de K-Means es mover los centroides de tal manera que esta suma total de distancias al cuadrado sea la menor posible. Un valor de inercia bajo significa que logramos ubicar los centroides de forma muy eficiente, muy cerca de sus respectivos puntos de datos.

La Definición Formal: WCSS

El término técnico para la inercia es WCSS (Within-Cluster Sum of Squares) o "Suma de Cuadrados Intra-Cluster". La fórmula es:

$$
\text{Inercia (WCSS)} = \sum_{j=1}^{k} \sum_{i \in C_j} \| x_i - \mu_j \|^2
$$

In [ ]:
kmeans.inertia_

#### Método del Codo (Elbow Method)

El método del codo **se basa enteramente en cómo se comporta la inercia a medida que aumentamos el número de clusters (k)**.

- La inercia siempre disminuye con más clusters: Esto es lógico. En el caso extremo, si el número de clusters es igual al número de datos, la distancia de cada dato a su centroide sería cero y la inercia sería 0.

- Por la razón anterior, no podemos simplemente elegir el k que nos dé la menor inercia. Lo que buscamos es el "codo": el punto a partir del cual añadir un nuevo cluster ya no reduce la inercia de forma significativa. Es el punto donde el beneficio de añadir un nuevo cluster es marginal, lo que nos sugiere que hemos encontrado el número de grupos "naturales" en los datos.

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X_scaled_df)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Método del Codo')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('WCSS')
plt.show()

El "codo" es claramente visible en k=5, confirmando nuestra intuición visual.

#### Coeficiente de Silueta (Silhouette Score)

La idea central del Coeficiente de Silueta es responder dos preguntas para cada punto de dato individual:

- ¿Qué tan bien encaja este punto en su propio cluster? (Cohesión)

- ¿Qué tan mal encajaría este punto en el siguiente cluster más cercano? (Separación)

Un buen clustering es aquel en el que, para la mayoría de los puntos, la respuesta a la primera pregunta es "muy bien" y la respuesta a la segunda es "muy mal".

Para cada punto de dato i, calculamos dos valores:

- a(i): **Cohesión Interna**. Es la distancia promedio de i a todos los demás puntos dentro del mismo cluster. Un valor bajo de a(i) es bueno, significa que el punto está en un vecindario denso y coherente.

- b(i): **Separación Externa**. Es la distancia promedio de i a todos los puntos en el siguiente cluster más cercano. El "siguiente cluster más cercano" es aquel cuya distancia promedio desde i es la más baja entre todos los clusters de los que i no es miembro. Un valor alto de b(i) es bueno, significa que los otros clusters están lejos.

Una vez que tenemos a(i) y b(i) para un solo punto i, la fórmula del coeficiente de silueta para ese punto es:

$$
s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$

El Numerador (b(i) - a(i)): Es el corazón del cálculo. Queremos que la separación b(i) sea lo más grande posible y que la cohesión a(i) sea lo más pequeña posible. Por lo tanto, un valor grande y positivo en el numerador es ideal.

El Denominador max(a(i), b(i)): Es un factor de normalización. Divide el resultado por el valor más grande entre a(i) y b(i) para asegurar que el puntaje final siempre esté entre -1 y 1.

- **Puntaje cercano a +1** (Ideal ✨): El punto está muy bien agrupado. Su propio cluster es muy denso y está muy lejos del siguiente cluster más cercano.

- **Puntaje cercano a 0** (Ambiguo 😐): El punto está en la frontera entre dos clusters. Está casi a la misma distancia de su propio cluster que del vecino. Esto indica que los clusters se superponen o que el punto podría pertenecer a cualquiera de los dos.

- **Puntaje cercano a -1** (Pésimo ❌): El punto probablemente ha sido asignado al cluster equivocado. En promedio, está más cerca de los puntos de un cluster vecino que de los puntos de su propio cluster. Es una fuerte señal de que la estructura del clustering es incorrecta.

In [ ]:
# Se define un rango de valores para el número de clusters (k) que vamos a probar.
# Empezamos en 2 porque el coeficiente de silueta no se puede calcular para un solo cluster (k=1).
# Probaremos desde k=2 hasta k=10.
range_n_clusters = range(2, 11)

silhouette_avg_scores = []

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    cluster_labels = clusterer.fit_predict(X_scaled_df)
    silhouette_avg = silhouette_score(X_scaled_df, cluster_labels)
    silhouette_avg_scores.append(silhouette_avg)
    print(f"Para n_clusters = {n_clusters}, el coeficiente de silueta promedio es: {silhouette_avg:.4f}")

plt.figure(figsize=(10, 5))
plt.plot(range_n_clusters, silhouette_avg_scores, marker='o', linestyle='--')
plt.title('Coeficiente de Silueta para Varios k')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Coeficiente de Silueta Promedio')
plt.show()

El coeficiente de silueta más alto también se obtiene para k=5.

 **Nota:**

 Un puntaje promedio cercano a 1 indica que los clusters son densos y están bien separados. Si es cercano a 0 o negativo, la estructura es débil.

In [ ]:
# Aplicar K-Means con k=5
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=42)
clusters_kmeans = kmeans.fit_predict(X_scaled_df)


In [ ]:
# Visualizar los clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Annual_Income', y='Spending_Score', hue='Cluster_KMeans', data=df, palette='viridis', s=100)

# Visualizar los centroides
centroids = scaler.inverse_transform(kmeans.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1], s=300, c='red', marker='X', label='Centroides')

plt.title('Segmentación de Clientes con K-Means')
plt.xlabel('Ingreso Anual (en miles de $)')
plt.ylabel('Puntuación de Gasto (1-100)')
plt.legend()
plt.show()

### Interpretación de los Clusters (¡La parte más importante!)

Ahora, le damos un significado a cada grupo:

- **Cluster 0 (Púrpura): Promedio.** Ingresos y gastos medios. Representan al cliente general del centro comercial. Estrategia: Promociones generales, programas de lealtad.

- **Cluster 1 (Azul Oscuro): Objetivo Principal.** ¡El grupo soñado! Ingresos altos y puntuación de gasto alta. Son los clientes más rentables. Estrategia: Programas VIP, acceso anticipado a productos, marketing de lujo.

- **Cluster 2 (Azul claro): Jóvenes Entusiastas.** Ingresos bajos, pero gastan mucho. Probablemente jóvenes, sensibles a las tendencias y ofertas. Estrategia: Marketing en redes sociales, descuentos por tiempo limitado.

- **Cluster 3 (Verde): Ahorradores Prudentes.** Ingresos altos, pero puntuación de gasto baja. Son clientes valiosos pero difíciles de atraer. Estrategia: Marketing enfocado en calidad, durabilidad e inversión a largo plazo.

- **Cluster 4 (Amarillo): Cautelosos.** Bajos ingresos y bajo gasto. Son muy sensibles al precio. Estrategia: Promociones de "compre uno, lleve otro", cupones de descuento.

In [ ]:
df.groupby('Cluster_KMeans')[['Annual_Income', 'Spending_Score', 'Age']].mean()

## ¿Podemos usar clustering para predecir algo? Por ejemplo, ¿predecir a qué cluster pertenecerá un nuevo cliente?

Una vez que has creado tus clusters y estás satisfecho con ellos, la etiqueta del cluster (Cluster_KMeans en nuestro notebook) se convierte en una nueva variable objetivo. Luego, puedes entrenar un modelo de clasificación (como una regresión logística o un árbol de decisión) para predecir la probabilidad de que un nuevo cliente, basándose en su edad, ingreso, etc., pertenezca a uno de tus segmentos definidos. Esto es extremadamente útil para la toma de decisiones en tiempo real.

## Clustering Avanzado: Manejo de Variables Categóricas con K-Prototypes

Hasta ahora, hemos trabajado principalmente con variables numéricas (Annual_Income, Spending_Score). Pero, ¿qué pasa con la columna Gender? ¿Podemos incluirla en nuestro análisis para obtener segmentos aún más ricos?

Los algoritmos como K-Means, que se basan en la distancia euclidiana, no pueden manejar texto como "Male" o "Female" directamente. Para resolver esto, podemos emplear modelos como K-Prototypes para datos mixtos.

In [ ]:
#!pip install kmodes

In [ ]:
# Importamos los nuevos algoritmos
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

K-Prototypes, en lugar de "centroides", crea "prototipos" de clusters que son un híbrido:

- Para las columnas numéricas, el prototipo es la media.

- Para las columnas categóricas, el prototipo es la moda.

La distancia total es una combinación ponderada de la distancia euclidiana y la de Hamming.

**Nota: Distancia de Hamming**

Se cuenta el número de variables en las que son diferentes dos observaciones: Un cliente ['Móvil', 'Chrome', 'Sí'] tendría una distancia de 0 con otro cliente idéntico, y una distancia de 3 con un cliente ['PC', 'Firefox', 'No'].


Vamos a segmentar a nuestros clientes usando Age, Annual_Income, Spending_Score y, ahora también, Gender.

In [ ]:
# Hacemos una copia del DataFrame para este análisis
df_mixed = df[['Age', 'Annual_Income', 'Spending_Score', 'Gender']].copy()
df_mixed

In [ ]:
# El algoritmo necesita saber la posición de las columnas categóricas.
# En nuestro caso, 'Gender' está en la última posición (índice 3).
categorical_columns_index = [3]

# Convertimos los datos a una matriz numpy
data_matrix = df_mixed.values

# K-Prototypes es sensible a la escala, por lo que estandarizamos las variables numéricas primero
# Nota: Estandarizamos solo las columnas numéricas (0, 1, 2)
scaler = StandardScaler()
data_matrix[:, 0:3] = scaler.fit_transform(data_matrix[:, 0:3])

# La columna 'Gender' también debe ser codificada numéricamente para el algoritmo
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_matrix[:, 3] = le.fit_transform(data_matrix[:, 3])
print(f"El mapeo de clases es: {le.classes_}")

# Aplicamos K-Prototypes
kproto = KPrototypes(n_clusters=5, init='Cao', n_init=5, verbose=0, random_state=42)
clusters_kproto = kproto.fit_predict(data_matrix, categorical=categorical_columns_index)
clusters_kproto

In [ ]:
# Añadimos los clusters al DataFrame original
df_mixed['Cluster_KPrototypes'] = clusters_kproto
df_mixed

In [ ]:
print(kproto.cluster_centroids_)

In [ ]:
# Analicemos el perfil de los clusters con los datos originales
print("\nPerfil de los clusters (medias y modas):")
df_mixed.groupby('Cluster_KPrototypes').agg({
    'Age': 'mean',
    'Annual_Income': 'mean',
    'Spending_Score': 'mean',
    'Gender': lambda x: x.mode()[0]  # Calculamos la moda para el género
}).round(2)

Ahora tenemos una segmentación mucho más rica:

- Cluster 0: Hombres de mediana edad, Cautelosos. Edad promedio mediana (~41 años), ingresos altos y gastos bajos.

- Cluster 1: Mujeres Jóvenes, Alto Poder Adquisitivo (Objetivo Principal). Edad promedio ~33 años, ingresos y gastos muy altos. Este es el segmento premium.

- Cluster 2: Mujeres de Mediana Edad, Ahorradoras. Edad promedio ~46 años, bajos ingresos y bajo gasto.

- Cluster 3: Mujeres muy Jóvenes, Entusiastas. Edad promedio muy baja (~25 años), bajos ingresos pero alto gasto.

- Cluster 4: Mujeres de Mediana Edad, Promedio. Edad promedio mediana (~55 años), ingresos y gastos en el rango medio.







In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Annual_Income', y='Spending_Score', hue='Cluster_KPrototypes', data=df_mixed, palette='tab10', s=100)
# 1. Extraemos solo la parte numérica de los prototipos (Age, Annual_Income, Spending_Score)
# kproto.cluster_centroids_ contiene tanto las medias numéricas como las modas categóricas.
# Seleccionamos las primeras 3 columnas que corresponden a nuestras variables numéricas.
numeric_prototypes = kproto.cluster_centroids_[:, 0:3]

# 2. Aplicamos la transformación inversa para devolver los centroides a su escala original.
# Usamos el mismo objeto 'scaler' que utilizamos para entrenar el modelo.
prototypes_original_scale = scaler.inverse_transform(numeric_prototypes)

# 3. Graficamos los prototipos (centroides) sobre el scatter plot.
# Extraemos las coordenadas correspondientes a 'Annual_Income' (índice 1) y 'Spending_Score' (índice 2).
plt.scatter(
    prototypes_original_scale[:, 1],  # Coordenada X: Ingreso Anual
    prototypes_original_scale[:, 2],  # Coordenada Y: Puntuación de Gasto
    s=300,                            # Tamaño del marcador
    c='red',                          # Color
    marker='X',                       # Forma del marcador
    label='Prototipos'                # Etiqueta para la leyenda
)

# Actualizamos la leyenda para incluir los prototipos
plt.legend()

plt.show()

### Búsqueda del K óptimo para K-Prototypes: se utiliza el método del codo

In [ ]:
# Lista para almacenar los costos para cada valor de k
costs = []
# Rango de clusters que vamos a probar
range_n_clusters = range(2, 11)

print("Calculando el costo para cada valor de k...")

# Bucle para probar diferentes números de clusters
for k in range_n_clusters:
    # Creamos una instancia de KPrototypes para el k actual
    kproto = KPrototypes(n_clusters=k, init='Cao', n_init=5, verbose=0, random_state=42)

    # Entrenamos el modelo con nuestros datos mixtos y especificamos las columnas categóricas
    kproto.fit(data_matrix, categorical=categorical_columns_index)

    # Guardamos el costo (la 'inercia' de K-Prototypes) en nuestra lista
    costs.append(kproto.cost_)

    print(f"Costo para k={k}: {kproto.cost_:.2f}")

# Graficamos el método del codo
plt.figure(figsize=(10, 6))
plt.plot(range_n_clusters, costs, marker='o', linestyle='--')
plt.title('Método del Codo para K-Prototypes')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Costo del Clustering')
plt.xticks(range_n_clusters)
plt.grid(True)
plt.show()